In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import trade_data
from passivbot import load_settings
from backtester import *

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [ ]:
settings = load_settings('default')
settings

In [ ]:
# modify ema spans
'''
p = 1.65
max_span = 60 * 18
n = 11
spans = np.array([p**i for i in range(n)])
spans = list((spans / spans.max() * max_span).round().astype(int))
spans
'''

In [ ]:
coins = sorted(set(settings['coins_long'] + settings['coins_shrt']))
symbols = [f'{c}/BTC' for c in coins]
symbols = sorted(symbols)
n_days = 30 * 18
symbols

In [ ]:
fee = 1 - 0.0675 * 0.01 # vip1

# modify settings here

#settings['hours_rolling_small_trade_window'] = 1.0
#settings['ema_spans_minutes'] = spans
#settings['profit_pct'] = 0.0025

#settings['account_equity_pct_per_trade'] = 0.0003

#settings['min_seconds_between_same_side_entries'] = 60

#settings['max_memory_span_days'] = 3600

#settings['coins_long'] = coins
#settings['coins_shrt'] = coins

#settings['exponent'] = 12
#settings['min_big_trade_cost_multiplier'] = 10

#settings['ema_spans_minutes'] = [15, 25, 40, 64, 102, 164, 263, 421, 675, 1080]
print(settings['account_equity_pct_per_trade'] * 60)

In [ ]:
# set no_download to True to load faster if already downloaded ohlcvs
high_low_means = load_hlms(symbols, n_days, no_download=False)

In [ ]:
df = add_emas(high_low_means, settings['ema_spans_minutes'], 0.005)
print(f'n_days {(df.index[-1] - df.index[0]) / 1000 / 60 / 60 / 24:2f}')

In [ ]:
df

In [ ]:
balance_list, lentr, sentr, lexit, sexit, lexitpl, sexitpl = backtest(df, settings)

In [ ]:
# balances are in terms of quot
bldf = pd.DataFrame([e for e in balance_list[::10]]).set_index('timestamp')

In [ ]:
bldfc = bldf.drop(['acc_equity_quot', 'acc_debt_quot', 'BTC'], axis=1)
bldfc.plot()
print('\nbiggest shrt bags')
print(bldfc.min().sort_values())
print('\nbiggest long bags')
print(bldfc.max().sort_values()[::-1])

In [ ]:
bldf.BTC.plot()

In [ ]:
start_equity = bldf.acc_equity_quot.iloc[0]
end_equity = bldf.acc_equity_quot.iloc[-1]
n_days = (bldf.index[-1] - bldf.index[0]) / 1000 / 60 / 60 / 24
avg_daily_gain = (end_equity / start_equity)**(1 / n_days)
hwme = bldf.acc_equity_quot.max()
hwmi = bldf.acc_equity_quot.idxmax()
n_days_hwm = (hwmi - bldf.index[0]) / 1000 / 60 / 60 / 24
max_daily_gain = (hwme / start_equity)**(1 / n_days_hwm)
plt.plot(np.linspace(0, 100, len(bldf)), bldf.acc_equity_quot.values)


print('               ndays', round(n_days, 2))
print('start end daily gain', round(avg_daily_gain, 8))
print('      max daily gain', max_daily_gain)
print('      low water mark', bldf.acc_equity_quot.min())
print('     high water mark', hwme)
print('                 end', bldf.acc_equity_quot.iloc[-1])
print('                mean', bldf.acc_equity_quot.mean())
print('         yearly gain', max_daily_gain ** 365)

xs = np.linspace(0, 100, len(bldf))
plt.plot(xs, bldf.acc_equity_quot.values)
wema = bldf.acc_equity_quot.ewm(span=10000).mean().values
plt.plot(xs, wema)
print('            ema gain', (wema[-1] / wema[0])**(1 / n_days))



In [ ]:
bldf.acc_debt_quot.plot()

In [ ]:
# bag status at end
sorted([(round(e[1], 6), e[0]) for e in dict(bldf.iloc[-1]).items() if 'acc' not in e[0]])

In [ ]:
r = {}
for s in map(lambda x: x.replace('/', '_'), symbols):
    # all buys
    try:
        abdf = pd.DataFrame(sorted(lentr[s] + sexit[s], key=lambda x: x['timestamp'])).set_index('timestamp')
    except:
        print(s, 'no buys')
        abdf = None
    # all sels
    try:
        asdf = pd.DataFrame(sorted(sentr[s] + lexit[s], key=lambda x: x['timestamp'])).set_index('timestamp')
    except:
        print(s, 'no sels')
        asdf = None
    if abdf is None or asdf is None:
        continue
    buy_cost = (abdf.price * abdf.amount).sum()
    sel_cost = (asdf.price * asdf.amount).sum()
    sum_cost = buy_cost + sel_cost
    buy_vwap = buy_cost / abdf.amount.sum()
    sel_vwap = sel_cost / asdf.amount.sum()
    diff = (sel_vwap / buy_vwap) * fee**2
    approx_profit = buy_cost * (diff * fee**2 - 1)
    r[s] = {'vol': sum_cost, 'svwap / bvwap': sel_vwap / buy_vwap,
            'vol * (sel_vwap / buy_vwap * fee**2 - 1)': sum_cost * (sel_vwap / buy_vwap * fee**2 - 1)}

sum_vol = sum(r[s]['vol'] for s in r)
print('sum vol', sum_vol)
print('monthly vol', sum_vol / n_days * 30)
rdf = pd.DataFrame(r).T.sort_values('vol * (sel_vwap / buy_vwap * fee**2 - 1)')
print('sum', rdf['vol * (sel_vwap / buy_vwap * fee**2 - 1)'].sum())
rdf

In [ ]:
# analyze particular symbol
s = 'XMR_BTC'

In [ ]:
try:
    ledf = pd.DataFrame(lentr[s]).set_index('timestamp')
    lexitdf = pd.DataFrame(lexit[s]).set_index('timestamp')
    lexitpdf = pd.DataFrame(lexitpl[s]).set_index('timestamp')
    did_long = True
except:
    print('no longs', s)
    did_long = False
try:
    sedf = pd.DataFrame(sentr[s]).set_index('timestamp')
    sexitdf = pd.DataFrame(sexit[s]).set_index('timestamp')
    sexitpdf = pd.DataFrame(sexitpl[s]).set_index('timestamp')   
    did_shrt = True
except:
    print('no shorts', s)
    did_shrt = False

In [ ]:
# longest bag durations
d = 1000 * 60 * 60 * 24
if did_long:
    for e in sorted(np.diff(np.append(lexitdf.index, ledf.index[-1]) / d))[-10:]:
        print('long bag duration days', round(e, 2))
print()
if did_shrt:
    for e in sorted(np.diff(np.append(sexitdf.index, sedf.index[-1]) / d))[-10:]:
        print('shrt bag duration days', round(e, 2))

In [ ]:
if did_long:
    high_low_means[s + '_mean'].plot()
    lexitpdf.price.plot(style='r-')
    ledf.price.plot(style='b.-')
    lexitdf.price.plot(style='ro')

In [ ]:
if did_shrt:
    high_low_means[s + '_mean'].plot()
    sexitpdf.price.plot(style='b-')
    sedf.price.plot(style='r.-')
    sexitdf.price.plot(style='bo')

In [ ]:
xs = np.linspace(1.0, 1.5, 100)
ys = [max(1.0, min(9.0, k**12)) for k in xs]
plt.plot(xs, ys)